In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import re
import math

In [2]:
first_url = 'https://www.glassdoor.com/Reviews/Revolut-Reviews-E1176471.htm?filter.iso3Language=eng'
pwd = os.getcwd()
driver_path = pwd + "/chromedriver.exe"
wd = webdriver.Chrome(executable_path=driver_path)
wd.get(first_url)

soup = bs(wd.page_source, 'html.parser')
total_reviews = re.findall(r"\d+[^\s]", soup.find(class_='css-ntu5m7 col-6 my-0').text.strip())[0]

wd.quit()

C:\Users\u10054206\AppData\Local\Temp\ipykernel_20416\2413263014.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=driver_path)


In [3]:
total_pages = math.ceil(int(total_reviews) / 10)
total_pages

83

In [18]:
ratings = []
employee_status = []
review_titles = []
review_dates = []
review_pros = []
review_cons = []
num_people_helpful = []
for i in range(total_pages):
    print(i)
    page_number = str(i)
    url = f'https://www.glassdoor.com/Reviews/Revolut-Reviews-E1176471_P{page_number}.htm?filter.iso3Language=eng'
    pwd = os.getcwd()
    driver_path = pwd + "/chromedriver.exe"
    wd = webdriver.Chrome(executable_path=driver_path)

    wd.get(url)

    time.sleep(2)

    soup = bs(wd.page_source, 'html.parser')

    for review in soup.find_all(class_='gdReview'):
        ratings.append(review.find(class_='ratingNumber mr-xsm').text.strip())

        employee_status.append(review.find(class_='pt-xsm pt-md-0 css-1qxtz39 eg4psks0').text.strip())

        review_titles.append(review.find(class_='mb-xxsm mt-0 css-93svrw el6ke055').text.strip())

        review_dates.append(review.find(class_='authorJobTitle middle common__EiReviewDetailsStyle__newGrey').text.strip())

        review_pros.append(review.find("span", {"data-test": "pros"}).text.strip().replace("\r", "").replace("\n", ""))

        review_cons.append(review.find("span", {"data-test": "cons"}).text.strip().replace("\r", "").replace("\n", ""))

        try:
            num_people_helpful.append(re.findall(r"\d+", review.find(class_="common__EiReviewDetailsStyle__socialHelpfulcontainer pt-std").text.strip())[0].replace("\xa0", ""))
        except IndexError as e:
            num_people_helpful.append(0)
    wd.quit()

0


C:\Users\u10054206\AppData\Local\Temp\ipykernel_20416\440619541.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path=driver_path)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82


In [21]:
df = pd.DataFrame(data={"Rating":ratings, "Employee Status":employee_status, "Review Title":review_titles, "Date":review_dates,
                    "Pros":review_pros, "Cons":review_cons, "Num People Helpful":num_people_helpful})
df

,Rating,Employee Status,Review Title,Date,Pros,Cons,Num People Helpful
0,5.0,Current Employee,Review,"Apr 26, 2022 - Anonymous Employee",Learning new content. Extremely detailed train...,Amount of initial information can be overwhelm...,0
1,1.0,"Current Employee, less than 1 year",horrible,"Apr 25, 2022 - Account Executive",do not work here. please find another job.,worst place to work. should not join.,5
2,5.0,"Former Employee, more than 1 year",-,"Mar 15, 2022 - Anonymous Employee",Innovative and strong ecosystem within teams,Great experience and not a lot of issues,0
3,4.0,"Current Employee, less than 1 year",Recruiter,"Mar 7, 2022 - Recruiter",Work culture is very good,"UNREASONABLE TARGETS , deadlines are crazy",0
4,4.0,"Current Employee, more than 1 year",Great Place to Work,"Mar 2, 2022 - Senior Software Engineer",- Huge growth opportunities to tech leadership...,- Startups environment: a lot of things change...,0
...,...,...,...,...,...,...,...
825,3.0,Current Employee,Exciting growth but bad culture,"Jan 5, 2018 - Anonymous Employee","Growth was really fast, fun team, cool product.","Bad working culture, staff not really respected.",4
826,3.0,"Former Employee, more than 1 year",Awful company to work for,"Nov 26, 2017 - Analyst",Its headquartered in Canary Wharf,Poor salaryNo strategyunorganised start up,14
827,1.0,"Former Employee, more than 1 year",Consider carefully,"Dec 14, 2017 - Anonymous Employee",All you can drink coffee/tea/diet coke,Poor managementLack of leadership and directio...,32
828,4.0,"Current Employee, less than 1 year",its ok to work here,"Oct 11, 2017 - Test Automation Engineer",Interesting projects and good compensations.,Really bag management. Very uncomfortable sit...,6


In [22]:
df.to_csv('reviews.csv', sep=';', index=False)